# Accuracy across 20 rows at 10M and 100M paths

In [ ]:

"""

Checks:
1. Statistical significance between 10M and 100M runs using MC-SE of the *mean*.
2. Whether the two runs match to 3 dp (|Δmean| < tol).
3. Whether each run’s own SE is small enough (SE_of_mean < tol).
"""

import re
import pandas as pd
import numpy as np
from math import sqrt

# ── edit these paths if needed ──────────────────────────────────
FILE_10M  = "results_10M.parquet"
FILE_100M = "results_100M.parquet"

# ── knobs ───────────────────────────────────────────────────────
ABS_TOL = 5e-4       # tolerance on mean difference for 3dp
Z_CRIT  = 1.96       # two-tailed z-critical for α = 0.05


def load_dfs():
    df10 = pd.read_parquet(FILE_10M)
    df100 = pd.read_parquet(FILE_100M)
    return df10, df100


def metrics_from_df(df):
    """
    Identify all metrics that have a matching SE column in the dataframe.
    """
    metrics = set()
    for col in df.columns:
        m = re.match(r"(.+)_se(?:_(\d+))?", col)
        if m:
            prefix, idx = m.group(1), m.group(2)
            metric = f"{prefix}_{idx}" if idx else prefix
            metrics.add(metric)
    return sorted(metrics)


def compute_run_se_of_mean(se_series):
    """
    Compute SE of the mean across rows:
      SE_mean = sqrt(sum(se_i**2)) / N_rows
    """
    return np.sqrt((se_series.values**2).sum()) / len(se_series)


def compare(df_a, df_b, tol=ABS_TOL, zcrit=Z_CRIT):
    mets = metrics_from_df(df_a)
    rows = []
    for m in mets:
        # determine se column name
        if '_' in m:
            base, idx = m.rsplit('_',1)
            se_col = f"{base}_se_{idx}"
        else:
            se_col = f"{m}_se"

        try:
            # compute means
            ma = df_a[m].mean()
            mb = df_b[m].mean()
            # compute SE of mean
            sea = compute_run_se_of_mean(df_a[se_col])
            seb = compute_run_se_of_mean(df_b[se_col])
        except Exception:
            # missing columns or other error: skip
            continue

        # skip if any is not finite
        if not all(np.isfinite([ma, mb, sea, seb])):
            continue

        # z-test for mean difference
        diff       = ma - mb
        pooled_se  = sqrt(sea**2 + seb**2)
        zscore     = diff / pooled_se
        significant= abs(zscore) > zcrit

        # 3dp agreement
        same_3dp   = abs(diff) < tol
        # intrinsic accuracy
        acc_a      = sea < tol
        acc_b      = seb < tol

        rows.append({
            "metric":         m,
            "mean_10M":       ma,
            "se_mean_10M":    sea,
            "mean_100M":      mb,
            "se_mean_100M":   seb,
            "diff":           diff,
            "pooled_se":      pooled_se,
            "z":              zscore,
            "significant?":   significant,
            "same_3dp?":      same_3dp,
            "accurate_10M?":  acc_a,
            "accurate_100M?": acc_b,
        })
    return pd.DataFrame(rows).set_index("metric")


def main():
    df10, df100 = load_dfs()
    cmp = compare(df10, df100)

    # PRICE diagnostics
    print("\nPRICE diagnostics:\n")
    print(cmp.loc["price", [
        "mean_10M","se_mean_10M",
        "mean_100M","se_mean_100M",
        "diff","pooled_se","z",
        "significant?","same_3dp?",
        "accurate_10M?","accurate_100M?"
    ]])

    # Full comparison table
    pd.set_option("display.float_format", "{:0.6f}".format)
    print("\nFull comparison table:\n")
    print(cmp)

    # Accuracy summary
    print("\nACCURACY SUMMARY:\n")
    match3 = cmp.index[cmp["same_3dp?"]].tolist()
    fail3  = cmp.index[~cmp["same_3dp?"]].tolist()
    sig    = cmp.index[cmp["significant?"]].tolist()
    acc10  = cmp.index[cmp["accurate_10M?"]].tolist()
    acc100 = cmp.index[cmp["accurate_100M?"]].tolist()

    print(f"Metrics matching 3dp       : {', '.join(match3) or 'None'}")
    print(f"Metrics failing  3dp       : {', '.join(fail3) or 'None'}")
    print(f"Significantly different    : {', '.join(sig) or 'None'}")
    print(f"Accurate at 3dp (10M run)  : {', '.join(acc10) or 'None'}")
    print(f"Accurate at 3dp (100M run) : {', '.join(acc100) or 'None'}")

if __name__ == "__main__":
    main()



PRICE diagnostics:

mean_10M           20.575977
se_mean_10M         0.003528
mean_100M          23.098340
se_mean_100M        0.001233
diff               -2.522363
pooled_se           0.003737
z                -674.890099
significant?            True
same_3dp?              False
accurate_10M?          False
accurate_100M?         False
Name: price, dtype: object

Full comparison table:

         mean_10M  se_mean_10M  mean_100M  se_mean_100M      diff  pooled_se  \
metric                                                                         
delta_0  0.219971     0.000170   0.310645      0.000071 -0.090674   0.000184   
delta_1  0.149243     0.000325   0.108130      0.000067  0.041113   0.000332   
delta_2  0.107227     0.000104   0.096484      0.000034  0.010742   0.000109   
gamma_0 -4.162500     0.016171   4.431250      0.006797 -8.593750   0.017541   
gamma_2 -6.606250     0.006380  -5.940625      0.002079 -0.665625   0.006710   
price   20.575977     0.003528  23.098340      0

c:\Users\jacks\anaconda3\envs\tf-gpu\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


# Ferguson and Green Check

## Original Code

In [1]:
import numpy as np, torch
from make_worst_of import fg_sample, price_mc, SEED_BASE

# 1) Fix your seeds for full reproducibility
np.random.seed(SEED_BASE)
torch.manual_seed(SEED_BASE)

# 2) Draw one scenario and compute price + SE
params = fg_sample()
price, se = price_mc(
    params,
    n_paths= 100_000_000,
    n_steps=64,
    return_se=True
)

# 3) Define your accuracy thresholds (in absolute price‐error units)
thresholds = {
    "1 cent (0.01)":    0.01,
    "0.1 cent (0.001)": 0.001,
    "0.01 cent (0.0001)": 0.0001,
}

# 4) Print results
print(f"price = {price:.6f},  SE = {se:.6f}\n")
for label, tol in thresholds.items():
    status = "PASS" if se < tol else "FAIL"
    print(f"{label:15}: {status}  (SE = {se:.6f} {'<' if status=='PASS' else '>'} {tol:.6f})")


price = 54.250000,  SE = 0.002989

1 cent (0.01)  : PASS  (SE = 0.002989 < 0.010000)
0.1 cent (0.001): FAIL  (SE = 0.002989 > 0.001000)
0.01 cent (0.0001): FAIL  (SE = 0.002989 > 0.000100)


Able to replicate the Ferguson and Green of 1 cent accuracy with both 10 Mil and 100 Mil paths

## Variance Reduction Code

In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
make_worst_of_dataset_fast.py
  • multi-GPU, single-process Monte-Carlo in FP32 for stability
  • Owen-scrambled Sobol, antithetic, Brownian bridge
  • Control variate: sum of single-asset Black-Scholes calls
  • Chunk-safe up to 100 M paths on 4×12 GiB GPUs
  • Exports price & Greeks with sampling-error columns
"""

import os
import math
import argparse
import pathlib
import sys
import time

import numpy as np
import torch
import pyarrow as pa
import pyarrow.parquet as pq
from torch.distributions import Beta, Normal
from torch.quasirandom import SobolEngine

# ─────────────────────────── knobs ────────────────────────────
# Use FP32 to avoid NaNs
torch.set_default_dtype(torch.float32)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark       = True

# Problem constants
N_ASSETS   = 3
R_RATE     = 0.03
EPS_REL    = 1e-4
SEED_BASE  = 42
CHUNK_MAX  = 1_000_000   # flush Parquet every X rows
CHUNK_PATH = 1_000_000   # inner GPU chunk size for MC

# CUDA devices
NGPU    = torch.cuda.device_count()
DEVICES = [torch.device(f"cuda:{i}") for i in range(NGPU)]
if NGPU == 0:
    sys.exit("No CUDA GPU visible – aborting.")

# ───────────────── Sampler helpers ──────────────────────────

def cvine_corr(d, a=5.0, b=2.0):
    """C-vine correlation sampler in FP32"""
    beta = Beta(torch.tensor([a], device="cuda"), torch.tensor([b], device="cuda"))
    P = torch.eye(d, device="cuda")
    for k in range(d - 1):
        for i in range(k + 1, d):
            rho = 2 * beta.sample().item() - 1.0
            for m in range(k - 1, -1, -1):
                rho = rho * math.sqrt((1 - P[m,i]**2)*(1 - P[m,k]**2)) + P[m,i]*P[m,k]
            P[k,i] = P[i,k] = rho
    ev, evec = torch.linalg.eigh(P)
    P_corr = evec @ torch.diag(torch.clamp(ev, min=1e-6)) @ evec.T
    return P_corr


def fg_sample():
    z = np.random.normal(0.5, 0.5, N_ASSETS)
    return dict(
        S0=100 * np.exp(z),
        sigma=np.random.uniform(0.0, 1.0, N_ASSETS),
        T=(np.random.randint(1, 44)**2) / 252.0,
        rho=cvine_corr(N_ASSETS).cpu().numpy(),
        K=100.0,
        r=R_RATE
    )

# ──────────── Brownian bridge transformer ───────────────────

def brownian_bridge(increments):
    """Simple Brownian-bridge reordering"""
    order = [increments.shape[1] - 1] + list(range(increments.shape[1] - 1))
    return increments[:, order, :]

# ──────────── QMC + Antithetic path generator ────────────────

def generate_qmc_paths(m, n_steps, d, device):
    """Generate Sobol + antithetic normal increments with Brownian bridge."""
    engine = SobolEngine(d * n_steps, scramble=True)
    # draw half the uniforms, append antithetic, move to GPU
    u = engine.draw(m // 2, dtype=torch.float32)
    u = torch.cat([u, 1.0 - u], dim=0).to(device)
    # clamp to avoid exact 0/1 -> inf in icdf
    u = u.clamp(min=1e-6, max=1.0 - 1e-6)
    normals = Normal(0.,1.).icdf(u).view(m, n_steps, d)
    return brownian_bridge(normals)

# ──────────── Raw MC price (no CV), chunked ──────────────────

@torch.no_grad()
def price_mc(params, n_paths, n_steps, *, return_se=False):
    """ MC with control variate: raw price + CV correction """
    # 1) Raw MC payoff and SE
    disc, se = price_mc_raw(params, n_paths, n_steps)
    raw_price = torch.mean(disc).item()

    # 2) Monte Carlo estimate of control variate: sum of individual call payoffs
    per_gpu = n_paths // NGPU
    ctrl_vals = []
    for dev in DEVICES:
        for offset in range(0, per_gpu, CHUNK_PATH):
            chunk = min(CHUNK_PATH, per_gpu - offset)
            Z = generate_qmc_paths(chunk, n_steps, N_ASSETS, dev)
            # simulate terminal asset prices
            S0    = torch.tensor(params['S0'],    device=dev)
            sigma = torch.tensor(params['sigma'], device=dev)
            T     = torch.tensor(params['T'],     device=dev)
            rho   = torch.tensor(params['rho'],   device=dev)
            K = params['K']; r = params['r']
            dt    = T / n_steps
            mu    = (r - 0.5 * sigma**2)
            sig   = sigma
            chol  = torch.linalg.cholesky(rho)

            logS = torch.log(S0).expand(chunk, N_ASSETS).clone()
            sqrt_dt = math.sqrt(dt.item())
            for k in range(n_steps):
                dW = Z[:,k,:] @ chol.T
                logS = logS + mu * dt + sig * sqrt_dt * dW
            ST    = torch.exp(logS)  # [chunk, assets]
            calls = torch.clamp(ST - K, 0.).sum(dim=1)  # [chunk]
            ctrl_vals.append(calls.cpu())
    ctrl_all = torch.cat(ctrl_vals)  # length n_paths
    mc_ctrl   = ctrl_all.mean().item()

    # 3) Analytical expectation of sum-of-calls
    E_ctrl = sum(
        bs_call_price(params['S0'][j], params['K'], params['r'],
                      params['T'], params['sigma'][j])
        for j in range(N_ASSETS)
    )

    # 4) Control variate correction
    price_cv = raw_price + (E_ctrl - mc_ctrl)
    if return_se:
        return price_cv, se
    return price_cv

# ────────── main + quick test ──────────────────────────────
if __name__ == "__main__":
    np.random.seed(SEED_BASE)
    torch.manual_seed(SEED_BASE)
    params = fg_sample()
    price, se = price_mc(params, n_paths=100_000_000, n_steps=64, return_se=True)
    print(f"price = {price:.6f}, SE = {se:.6f}")


price = 32.144659, SE = 0.002985
